In [1]:
import torch
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import logging, os, argparse

from prompt_tuner import PromptTuner

2023-03-20 17:39:25.737070: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [4]:
# this substitutes argparse in Jupyter Notebook
class Args:
    model_name = 't5-base'
    task = 'boolq'
    batch_size = 4
    select_k_per_class = 500
    prefix_len = 10
    prefix_path = ''
    freeze_weights = 1
    freeze_except = 'none'
    lr = 0.5
    seq_len = 512
    early_stopping = 1
    prefix_MLP = 'MLP1'
    bottleneck_size = 500
    mlp_layer_norm = 1
    get_test_subset = False

args=Args()

In [5]:
prompt_tuner = PromptTuner(model_name=args.model_name,
                           task=args.task,
                           batch_size=args.batch_size,
                           select_k_per_class=args.select_k_per_class,
                           prefix_len=args.prefix_len,
                           prefix_path=args.prefix_path if args.prefix_path!='' else None, # path to the pre-trained progressive prompt
                           freeze_weights=args.freeze_weights==1,
                           freeze_except=args.freeze_except,
                           lr=args.lr,
                           weight_decay=1e-5,
                           seq_len=args.seq_len,
                           early_stopping=args.early_stopping==1,
                           prefix_MLP=args.prefix_MLP,
                           bottleneck_size=args.bottleneck_size, # bottleneck size in case of using MLP reparametrization
                           mlp_lr=None,
                           mlp_layer_norm=args.mlp_layer_norm==1,
                           weight_decay_mlp=None,
                           get_test_subset=args.get_test_subset,
                          )


Freezing weights
Using MLP reparametrization with bottleneck =  500
Using skip connection in MLP


/h/anastasia/anaconda3/envs/nlp_env/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Reusing dataset super_glue (/h/anastasia/.cache/huggingface/datasets/super_glue/boolq/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)


  0%|          | 0/1000 [00:00<?, ?ex/s]

/h/anastasia/anaconda3/envs/nlp_env/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:194: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/h/anastasia/anaconda3/envs/nlp_env/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you ca

k =  500   k-val =  -1


Reusing dataset super_glue (/h/anastasia/.cache/huggingface/datasets/super_glue/boolq/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7)
Loading cached shuffled indices for dataset at /h/anastasia/.cache/huggingface/datasets/super_glue/boolq/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7/cache-56923150e227aa8a.arrow
Loading cached processed dataset at /h/anastasia/.cache/huggingface/datasets/super_glue/boolq/1.0.2/d040c658e2ddef6934fdd97deb45c777b6ff50c524781ea434e7219b56a428a7/cache-261c94d7899cb575.arrow


In [6]:
prompt_tuner.prefix_MLP

ResMLP(
  (module): Sequential(
    (0): Linear(in_features=768, out_features=500, bias=True)
    (1): ReLU()
    (2): Linear(in_features=500, out_features=768, bias=True)
    (3): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
)

In [ ]:
p = prompt_tuner.model.prompt


In [ ]:
score_dict = prompt_tuner.train_one_task(epochs=4,
                                         eval_every_N=1,
                                         save_path='')

task =  boolq
Freezing all MLPs except for  boolq
0


  0%|          | 0/250 [00:00<?, ?it/s]

  0%|          | 0/818 [00:00<?, ?it/s]

['true', 'true', 'false', 'true']
['false', 'true', 'true', 'false']
['false', 'true', 'true', 'true']
['false', 'false', 'false', 'true']
['false', 'true', 'true', 'false']
['true', 'true', 'true', 'false']
['true', 'false', 'true', 'false']
['true', 'false', 'true', 'false']
['false', 'true', 'false', 'false']
['false', 'false', 'false', 'false']
['false', 'true', 'true', 'true']
['false', 'false', 'true', 'false']
['false', 'true', 'false', 'true']
['false', 'true', 'false', 'true']
['true', 'true', 'true', 'false']
['false', 'true', 'true', 'true']
['false', 'false', 'true', 'false']
['false', 'false', 'true', 'true']
['true', 'true', 'true', 'true']
['true', 'true', 'true', 'true']


In [8]:
score_dict

{'val': {'acc': [0.6840978593272171], 'loss': [0.3668331]},
 'train': {'acc': [0.498, 0.565, 0.627, 0.634, 0.647],
  'loss': [0.7972758, 0.4028198, 0.36044654, 0.34218577, 0.3419612]}}